In [59]:
# Shoutout to https://github.com/swar/nba_api

from nba_api.stats.endpoints import LeagueGameFinder
import pandas as pd
from pandasql import sqldf

pysqldf = lambda x: sqldf(x, globals())

In [75]:
# getting df of games for training data


# season_nullable requires format YYYY-YY
# date_from_nullable and date_to_nullable require the format MM/DD/YYYY

games = LeagueGameFinder(
   season_nullable = '2021-22'
#     , date_from_nullable = '02/02/2022',                                                     
#     date_to_nullable = '02/02/2022'
    ).league_game_finder_results.get_data_frame()

# filter to exclude G-league games using TEAM_ID
g_league_teams = [
    'Wisconsin Herd'
    , 'Lakeland Magic'
    , 'Agua Caliente Clippers'
    , 'Sioux Falls Skyforce'
    , 'Windy City Bulls'
    , 'Raptors 905'
    , 'Greensboro Swarm'
    , 'Delaware Blue Coats'
    , 'Memphis Hustle'
    , 'Iowa Wolves'
    , 'Texas Legends'
    , 'Stockton Kings'
    , 'Long Island Nets'
    , 'Cleveland Charge'
    , 'Fort Wayne Mad Ants'
    , 'Capital City Go-Go'
    , 'Santa Cruz Warriors'
    , 'Birmingham Squadron'
    , 'Westchester Knicks'
    , 'Motor City Cruise'
    , 'Rio Grande Valley Vipers'
    , 'Salt Lake City Stars'
    , 'South Bay Lakers'
    , 'Oklahoma City Blue'
    , 'Austin Spurs'
    , 'Maine Celtics'
    , 'G League Ignite'
    , 'Grand Rapids Gold'
    , 'College Park Skyhawks'
    , 'Ciudad de Mexico Capitanes'
]

games = games[~games['TEAM_NAME'].isin(g_league_teams)]


# There are two entries per game (one for each team), so we need to separate the two and join on GAME_ID,
# naming one table "team 1" and the other "team 2". Doing this with the help of pandasql and row_number().

query = '''
SELECT * FROM 
    (SELECT row_number() over(PARTITION BY GAME_ID) rownum, * FROM games)
WHERE rownum = {}
ORDER BY 1
'''
team1_games = pysqldf(query.format('1'))
team2_games = pysqldf(query.format('2'))
team1_games.columns = [c+'_team1' for c in team1_games.columns]
team2_games.columns = [c+'_team2' for c in team2_games.columns]
team1_games = team1_games.rename(columns={'GAME_ID_team1': 'GAME_ID'})
team2_games = team2_games.rename(columns={'GAME_ID_team2': 'GAME_ID'})

games = pd.merge(team1_games, team2_games, on='GAME_ID', how='inner')
games = games.rename(columns={'GAME_ID': 'game_id'})

# consolidating duplicate columns that are not specific to one team
consolidate_cols = ['SEASON_ID', 'GAME_DATE']
for c in consolidate_cols:
    games = games.drop(c + '_team1', axis=1)
    games = games.rename(columns={c+'_team2': c})

# We can use WL_team1 or WL_team2 as our label column.
query = 'SELECT *, CASE WHEN WL_team1 = "W" THEN 1 ELSE 0 END AS label FROM games'
games = pysqldf(query)

##### Features
##############

# Since the goal is to run the model before games happen, the features cannot contain any info that happens
# during the game. All features need to come from past games, as recent as the previously played game.
features = list()
# order by date and do lag and then running total of wins losses, so that for a feature you can do wins_team1 < wins_team2
# also get running totals of stats like points etc.
query = '''
    SELECT
        game_id
        , TEAM_NAME_team1
        , TEAM_NAME_team2
        , GAME_DATE
        , CASE WHEN WL_team1 = "W" THEN 1 ELSE 0 END AS team1_w_ind
        , CASE WHEN WL_team2 = "W" THEN 1 ELSE 0 END AS team2_w_ind
        , CASE WHEN WL_team1 = "L" THEN 1 ELSE 0 END AS team1_l_ind
        , CASE WHEN WL_team2 = "L" THEN 1 ELSE 0 END AS team2_l_ind
    FROM games
    ORDER BY 1
        '''
df = pysqldf(query)

query = '''
    SELECT
        game_id
        , LAG(team1_w_ind) OVER(ORDER BY TEAM_NAME_team1, GAME_DATE) 
        , LAG(team2_w_ind) OVER(ORDER BY TEAM_NAME_team2, GAME_DATE)
        , LAG(team1_l_ind) OVER(ORDER BY TEAM_NAME_team1, GAME_DATE)
        , LAG(team2_l_ind) OVER(ORDER BY TEAM_NAME_team2, GAME_DATE)
        , TEAM_NAME_team1
        , GAME_DATE
    FROM df
        '''
pysqldf(query)

,game_id,"LAG(team1_w_ind) OVER(ORDER BY TEAM_NAME_team1, GAME_DATE)","LAG(team2_w_ind) OVER(ORDER BY TEAM_NAME_team2, GAME_DATE)","LAG(team1_l_ind) OVER(ORDER BY TEAM_NAME_team1, GAME_DATE)","LAG(team2_l_ind) OVER(ORDER BY TEAM_NAME_team2, GAME_DATE)",TEAM_NAME_team1,GAME_DATE
0,0012100016,NaN,0.0,NaN,1.0,Atlanta Hawks,2021-10-06
1,0012100055,0.0,1.0,1.0,0.0,Atlanta Hawks,2021-10-14
2,0022100014,1.0,1.0,0.0,0.0,Atlanta Hawks,2021-10-21
3,0022100082,1.0,0.0,0.0,1.0,Atlanta Hawks,2021-10-30
4,0022100097,0.0,1.0,1.0,0.0,Atlanta Hawks,2021-11-01
...,...,...,...,...,...,...,...
841,0022100642,1.0,1.0,0.0,0.0,Washington Wizards,2022-01-15
842,0022100656,0.0,1.0,1.0,0.0,Washington Wizards,2022-01-17
843,0022100669,1.0,0.0,0.0,1.0,Washington Wizards,2022-01-19
844,0022100689,0.0,0.0,1.0,1.0,Washington Wizards,2022-01-21
